In [1]:
# Only want to use one gpu
import os
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID" # so the IDs match nvidia-smi
os.environ["CUDA_VISIBLE_DEVICES"] = "1" # "0, 1" for multiple

from tensorflow.python.client import device_lib

def get_available_gpus():
    local_device_protos = device_lib.list_local_devices()
    return [x.name for x in local_device_protos if x.device_type == 'GPU']

print(get_available_gpus())

from utils import load_data

seq_arr, test_seq_arr, labels, word_index, index_word, vs, embedding_matrix = load_data('word', 'glove')
seq_arr.shape, test_seq_arr.shape, embedding_matrix.shape

['/device:GPU:0']


Using TensorFlow backend.


((1099063, 30), (56370, 30), (59728, 300))

In [2]:
from utils import *
from keras import callbacks
from timeit import default_timer as timer
from keras import models, losses, metrics, layers, optimizers
from keras.utils import multi_gpu_model
import tensorflow as tf
import re
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
import os
import keras

import numpy as np
import pandas as pd

from tensorflow.python.client import device_lib

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'

datadir = '/home/wjk68/data/quora/'

def get_available_gpus():
    local_device_protos = device_lib.list_local_devices()
    return [x.name for x in local_device_protos if x.device_type == 'GPU']


get_available_gpus()

['/device:GPU:0']

## Word-Level Model

In [11]:
from keras.models import *
from keras.layers import *

sequence_input = Input(shape=(seq_arr.shape[1],), dtype='int32')

embedding_layer = Embedding(embedding_matrix.shape[0], 
                             embedding_matrix.shape[1],
                             weights = [embedding_matrix],
                             name = 'embedding')

embedded_sequences = embedding_layer(sequence_input)

l_cov1= Conv1D(128, 5, activation='relu')(embedded_sequences)
l_pool1 = MaxPooling1D(2)(l_cov1)
l_cov2 = Conv1D(128, 5, activation='relu')(l_pool1)
l_pool2 = MaxPooling1D(1)(l_cov2)
l_cov3 = Conv1D(128, 5, activation='relu')(l_pool2)
# l_pool3 = MaxPooling1D(2)(l_cov3)  # global max pooling
l_flat = Flatten()(l_cov3)
l_dense = Dense(128, activation='relu')(l_flat)
l_dense = Dropout(0.5)(l_dense)
preds = Dense(1, activation='sigmoid')(l_dense)
                          
model = Model(inputs = [sequence_input], output = [preds])

model.compile(optimizer = optimizers.Adam(), loss = losses.binary_crossentropy,
              metrics = [metrics.binary_crossentropy, metrics.binary_accuracy,
                         f1])

model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_9 (InputLayer)         (None, 30)                0         
_________________________________________________________________
embedding (Embedding)        (None, 30, 300)           17918400  
_________________________________________________________________
conv1d_22 (Conv1D)           (None, 26, 128)           192128    
_________________________________________________________________
max_pooling1d_17 (MaxPooling (None, 13, 128)           0         
_________________________________________________________________
conv1d_23 (Conv1D)           (None, 9, 128)            82048     
_________________________________________________________________
max_pooling1d_18 (MaxPooling (None, 9, 128)            0         
_________________________________________________________________
conv1d_24 (Conv1D)           (None, 5, 128)            82048     
__________

/home/wjk68/.local/lib/python3.6/site-packages/ipykernel_launcher.py:24: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=[<tf.Tenso..., outputs=[<tf.Tenso...)`


In [ ]:
history = model.fit(seq_arr, labels, validation_split = 0.4,
                    epochs = 25, batch_size = 1024)

Train on 659437 samples, validate on 439626 samples
Epoch 1/25
659437/659437 [==============================] - 25s 38us/step - loss: 0.1372 - binary_crossentropy: 0.1372 - binary_accuracy: 0.9512 - f1: 0.3988 - val_loss: 0.1104 - val_binary_crossentropy: 0.1104 - val_binary_accuracy: 0.9567 - val_f1: 0.5412
Epoch 2/25
659437/659437 [==============================] - 23s 34us/step - loss: 0.0991 - binary_crossentropy: 0.0991 - binary_accuracy: 0.9607 - f1: 0.6110 - val_loss: 0.1085 - val_binary_crossentropy: 0.1085 - val_binary_accuracy: 0.9575 - val_f1: 0.5695
Epoch 3/25
659437/659437 [==============================] - 23s 34us/step - loss: 0.0832 - binary_crossentropy: 0.0832 - binary_accuracy: 0.9661 - f1: 0.6820 - val_loss: 0.1146 - val_binary_crossentropy: 0.1146 - val_binary_accuracy: 0.9560 - val_f1: 0.5862
Epoch 4/25
659437/659437 [==============================] - 23s 35us/step - loss: 0.0632 - binary_crossentropy: 0.0632 - binary_accuracy: 0.9742 - f1: 0.7668 - val_loss: 0.15

In [ ]:
EMBEDDING_FILE = datadir + 'glove.840B.300d/glove.840B.300d.txt'
def get_coefs(word,*arr): 
    return word, np.asarray(arr, dtype='float32')
embeddings_index = dict(get_coefs(*o.split(" ")) for o in open(EMBEDDING_FILE))

In [ ]:
all_embs = np.stack(embeddings_index.values())
emb_mean,emb_std = all_embs.mean(), all_embs.std()
embed_size = all_embs.shape[1]

embedding_matrix = np.random.normal(emb_mean, emb_std, (vs, embed_size))

print('Embedding Matrix Shape: ', embedding_matrix.shape)

In [ ]:
from multiprocessing import Pool
from timeit import default_timer as timer
pool = Pool(15)

def get_embed_vector(word):
    return embeddings_index.get(word)

In [ ]:
words = word_index.keys()
r = pool.map(get_embed_vector, words)
pool.close()
pool.join()

# not_in = 0
# for word, i in word_index.items():
#     embedding_vector = embeddings_index.get(word)
#     if embedding_vector is not None: 
#         embedding_matrix[i] = embedding_vector
#     else:
#         not_in += 1

# print('There were ', not_in 'words not in the embeddings.')

In [ ]:
embedding_matrix = np.random.normal(emb_mean, emb_std, (vs, embed_size))

print('Embedding Matrix Shape: ', embedding_matrix.shape)

In [ ]:
arr = np.array(r)

In [ ]:
not_found = 0
for a in arr:
    if a is not None:
        continue
    else:
        not_found += 1

In [ ]:
not_found

In [ ]:
embedding_matrix = np.random.normal(size = (vs, 300))
for i, a in enumerate(arr):
    if a is not None:
        embedding_matrix[i] = a
    else:
        continue
        
embedding_matrix.shape

In [ ]:
np.save('glove_embeddings.npy', embedding_matrix)

In [ ]:
for i in r:
    if i == None:
        print('Found')

In [ ]:
np.mean(r == None)

In [ ]:
with open('word_index.json', 'w') as f:
    f.write(json.dumps(word_index))
    
with open('index_word.json', 'w') as f:
    f.write(json.dumps(index_word))

In [ ]:
sequences, labels, test_sequences, word_index, index_word, wc, vs = format_data(train_df, test_df,
                                                                            filters = '')
print('Shape of data: ', sequences.shape)
print('Shape of labels: ', labels.shape)
print('Number of words in vocab: ', vs)

In [ ]:
def sequence_to_query(sequence, index_word):
    query = [index_word.get(i, 'UNK') for i in sequence]
    return ' '.join(query)

In [ ]:
index_word[0]

In [ ]:
sequence_to_query(sequences[15], index_word)

In [ ]:
fasttext_embeddings = np.load('fasttext_embeddings.npy')
# fasttext_embeddings = normalize(fasttext_embeddings)